In [1]:
import os
import sys



In [2]:
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

Welcome to JupyROOT 6.24/07


/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-setuptools/60.9.3-ddcab3b6a742c72fed5554de8ccaaf18/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-setuptools/60.9.3-ddcab3b6a742c72fed5554de8ccaaf18/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
filelist =['/mnt/hadoop/se/store/user/milliqan/trees/v34/MilliQan_Run1190.155_v34.root:t']

filelist = []
directory = '/mnt/hadoop/se/store/user/milliqan/trees/v34/'
base_name = "MilliQan_Run1190"
for filename in os.listdir(directory):
    if filename.startswith(base_name) and filename.endswith(".root"):
        filelist.append(directory+filename)

In [4]:
branches = ['boardsMatched', 'height', 'layer','type','chan','pickupFlag']

In [ ]:
#define the milliqan cuts object
mycuts = milliqanCuts()

#example of changing cut value
#create cut so it can read the 1 NPE signal(36mV)
#with "branches=branches" it remove all pulses below 36mV
heightCutLowEphoton = mycuts.getCut(mycuts.heightCut, 'heightCutLowEphoton', cut=36, branches=branches)

#example of cutting on events
#event based cut
#fourLayerCut is a branch event based tag. But each pulse in a same event has same tag.
#with "cut = True", event being tagged as fail will be removed from the table.
fourLayerCut = mycuts.getCut(mycuts.fourLayerCut, 'fourLayerCut', cut=True)

barCut = mycuts.getCut(mycuts.barCut,'barCut', cut=True)

In [ ]:
#example of creating a combined cut
#I need to put the barCut into the event cut, since the bar the cut by de

#things inside ['fourLayerCut'] is used for checking t/f value inside the table. BTW the reason pickup and boardmaching is not here is that those two function has cut off the event being tagged as false in eight those two columns.
#this is done at the "branches = branch" section

eventCuts = mycuts.getCut(mycuts.combineCuts, 'eventCuts', ['fourLayerCut','bar','f'])


In [ ]:
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

In [ ]:
#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
nbars = r.TH1F("nbars", "n_BARS", 32, 0, 32)

#add root histogram to plotter

#in my function NBarHitsplot, there is no need to use any "variable".
#The "height" at here is just a place holder, so function milliqanPlot() doesn't crash
myplotter.addHistograms(nbars, 'height')

In [ ]:
#defining the cutflow
#bar cut is causing a bug
#cutflow = [boardMatchCut, pickupCut,heightCutLowEphoton,barCut,fourLayerCut, myplotter.dict['nbars']]
cutflow = [boardMatchCut, pickupCut,heightCutLowEphoton,fourLayerCut, myplotter.dict['nbars']]

In [ ]:


#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

In [ ]:
myiterator = milliqanProcessor(filelist, branches, myschedule, mycuts, myplotter,max_events=2)

In [ ]:
#run the milliqan processor
myiterator.run()

In [ ]:
c1 = r.TCanvas("c1", "c1", 400, 400)
c1.cd()
nbars.Draw()
c1.Draw()

In [ ]:
#demo
#define the necessary branches to run over
branches = ['boardsMatched', 'time_module_calibrated', 'height', 'area', 'column', 'row', 'layer']

#define the milliqan cuts object
mycuts = milliqanCuts()

#example of changing cut value
heightCut200 = mycuts.getCut(mycuts.heightCut, 'heightCut200', cut=500)

#example of cutting on events
fourLayerCut = mycuts.getCut(mycuts.fourLayerCut, 'fourLayerCut', cut=False)

#example of creating a combined cut
eventCuts = mycuts.getCut(mycuts.combineCuts, 'eventCuts', ['fourLayerCut', 'straightLineCut', 'heightCut200'])

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
h_height = r.TH1F("h_height", "Height", 140, 0, 1400)

#add root histogram to plotter
myplotter.addHistograms(h_height, 'height', 'eventCuts')



In [ ]:

#defining the cutflow
cutflow = [mycuts.layerCut, mycuts.fourLayerCut, mycuts.straightLineCut, heightCut200, eventCuts, myplotter.dict['h_height']]

In [ ]:

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, mycuts, myplotter, max_events=1000)

#run the milliqan processor
myiterator.run()

In [ ]:
#explore

for events in uproot.iterate(

            #files
            filelist,

            #branches
            branches,

            step_size=1000,

            num_workers=8,

            ):
    print(events)

In [ ]:
#learn iterate
for events in uproot.iterate(

            #files
            filelist,

            #branches
            branches,
            step_size=10
    
            ):
    #print(len(events))
    #print(events)
    #print(type(events))
    expanded_array = ak.to_pandas(events)
    print(expanded_array)
    print("next")
    break
    
    

In [ ]:
#learn iterate\
branches = ['boardsMatched', 'height', 'layer','type','chan','pickupFlag']
for events in uproot.iterate(

            #files
            filelist,

            #branches
            branches,
            step_size=1000
    
            ):
    
    
    
    #print(len(events))
    #print(events)
    intialdf = ak.to_pandas(events)
    print(intialdf)
    
    
    #print(type(events))
    for branch in branches:
        if branch == 'boardsMatched': continue
        events[branch] = events[branch][events.boardsMatched]
    afterdf = ak.to_pandas(events)
    #print(afterdf)
    
    for branch in branches:
        if branch == 'boardsMatched': continue
        events[branch] = events[branch][events.pickupFlag]
    
    
    lastdf = ak.to_pandas(events)
    #print(lastdf)
    
    
    #type cut
    barCut = events['type'] == 0 # create pulse base T/F table
    for branch in branches:
        if branch == 'boardsMatched': continue   #this is needed to avoid too many jacked slice in array.(dont know the meaing)
        events[branch] = events[branch][barCut]
    
    
    typpecutdf = ak.to_pandas(events)
    #print(typpecutdf)
    
    #print the key name for high level akw array
    field_names = events.fields
    #print(field_names)

    
    
    
    #print(type(events))
    #print(str(events))
    
    
    #pulse height cut 1 npe
    heightcut = events.height > 36
    #print(heightcut)
    
    
    for branch in branches:
        if branch == 'boardsMatched': continue
        events[branch] = events[branch][heightcut]
    
    heightCutdf = ak.to_pandas(events)
    #print(heightCutdf)
    
    
    #1 hit + per layer
    
    events['fourLayerCut'] = ak.any(events.layer==0, axis=1) & ak.any(events.layer==1, axis=1) & ak.any(events.layer==2, axis=1) & ak.any(events.layer==3, axis=1)
    #print(ak.to_pandas(events))#event based
    events = events[events['fourLayerCut']]
    print("final result")
    print(ak.to_pandas(events))
    
    
    
    
    
    #FLcut,events=ak.broadcast_arrays(FLcut,events)
    #print(FLcut)
    #print(events)
    #seems doesn't work
    #need to used broadcase
    
    #FLcut = events[events['fourLayerCut']]
    
    #print(events)
    #for branch in branches:
    #    if branch == 'boardsMatched': continue
    #    events = events[branch][FLcut]
    
    
        
    #FLcutdf = ak.to_pandas(events)
    #print(FLcutdf)
    

        
    #events["heightCut"] = events.height >= int(1200)  #pulse based table
    #events['fourLayerCut'] = ak.any(events.layer==0, axis=1) & ak.any(events.layer==1, axis=1) & ak.any(events.layer==2, axis=1) & ak.any(events.layer==3, axis=1) 
    #expanded_array = ak.to_pandas(events)
    #print(expanded_array)
    
    print("next")
    break

In [ ]:
diaplay(afterdf)

In [ ]:
display(expanded_array)